In [69]:
type automate = int * (int * int) array * bool array ;;

let a1 = 2, [|(1, 1) ; (0, 0)|], [|false; true|]
let a2 = 2, [|0, 1 ; 1, 0|], [|false; true|]  ;;
let a3 = 3, [|(1, 2); (1, 2); (2, 1)|], [|false; false; true|]
let a4 = 4, [|(1, 2) ; (0, 3); (3, 0); (2, 1)|], [|false ; false ; true ; true|]
let a5 = 3, [|(0, 1); (2, 0) ; (1, 0)|], [|false; false ; true|]

type automate = int * (int * int) array * bool array


val a1 : int * (int * int) array * bool array =
  (2, [|(1, 1); (0, 0)|], [|false; true|])


val a2 : int * (int * int) array * bool array =
  (2, [|(0, 1); (1, 0)|], [|false; true|])


val a3 : int * (int * int) array * bool array =
  (3, [|(1, 2); (1, 2); (2, 1)|], [|false; false; true|])


val a4 : int * (int * int) array * bool array =
  (4, [|(1, 2); (0, 3); (3, 0); (2, 1)|], [|false; false; true; true|])


val a5 : int * (int * int) array * bool array =
  (3, [|(0, 1); (2, 0); (1, 0)|], [|false; false; true|])


In [70]:
let numero n a =
  let t = Array.make n (-1) in
  let rec aux index = function
    |[] -> ()
    |h::q -> t.(h) <- index ; aux (index + 1) q in
  aux 0 a ; t ;;

val numero : int -> int list -> int array = <fun>


In [71]:
let etats_accessibles aut = 
  let n, delta, f = aut in 
  let visited = Array.make n false in
  let parcours = ref [] in
  let rec aux etat =
      if not visited.(etat) then 
          begin
          visited.(etat) <- true ;
          parcours := etat :: !parcours ;
          let succ_a, succ_b = delta.(etat) in
          aux succ_a ;
          aux succ_b
          end 
  in aux 0 ;
  List.rev !parcours ;;

val etats_accessibles : int * (int * int) array * 'a -> int list = <fun>


In [72]:
let partie_accessible aut = 
    let n, delta, f = aut in
    let new_etats = etats_accessibles aut in 
    let apparition = numero n new_etats in
    let new_n = List.length new_etats in
    let new_delta = Array.make new_n (0,0) in
    let new_f = Array.make new_n false in
    let rec aux = function
        |[] -> (new_n, new_delta, new_f)
        |h::t -> let s = apparition.(h) in
                new_f.(s) <- f.(h);
                let succ_a, succ_b = delta.(h) in
                new_delta.(s) <- apparition.(succ_a) ,apparition.(succ_b);
                aux t in
    aux new_etats ;;


val partie_accessible :
  int * (int * int) array * bool array ->
  int * (int * int) array * bool array = <fun>


In [73]:
let existe_morphismes aut1 aut2 =
    let n1, delta1, f1 = aut1 in 
    let n2, delta2, f2 = aut2 in
    let def = ref true in (*variable indiquant si un morphisme existe ou non*)
    let visited = Array.make n1 false in
    let morphisme = Array.make n1 (-1) in 
    let construire etat1 etat2 = (*fonction qui construit le morphisme*)
        if morphisme.(etat1) <> 1 (*si phi(q) ne s'est pas encore vu etre associe une image, on la definit*)
            then begin
                if ( f1.(etat1) = f2.(etat2) ) || ( (not f1.(etat1)) = (not f2.(etat2)) ) (*test de la condition (4)*)
                then morphisme.(etat1) <- etat2 (*si (4) est respectee, alors on peut definir phi(q)*)
                else def := false (*sinon il n'existe pas de morphisme*)
            end
        else if morphisme.(etat1) <> etat2 (*si phi(q) est deja defini, mais que la condition (3) n'est pas respectee...*)
             then def := false (*... alors il n'existe pas de morphisme*)
    in
    let rec aux etat1 = (*fonction qui parcours l'automate afin de constuire le morphisme*)
        if not visited.(etat1) (*on verifie que le sommet n'a pas deja ete visite*)
        then begin
            let etat2 = morphisme.(etat1) in
            let succ1_a, succ1_b = delta1.(etat1) in (*on construit le morphisme recursivement, en partant du sommet que l'on visite*)
            let succ2_a, succ2_b = delta2.(etat2) in
            visited.(etat1) <- true ;
            construire succ1_a succ2_a ; (*on definit les images de phi pour sigma = a*)
            construire succ1_b succ2_b ; (*on definit les images de phi pour sigma = b*)
            if !def then (aux succ1_a ; aux succ1_b) (*si le morphisme existe (i.e les etapes de construction se sont achevees), alors on continue jusqu'a ce que tous les sommets soient visites*)
            end 
    in
    morphisme.(0) <- 0 ; (*initialisation de la constuction avec la condition (2)*)
    aux 0 ; (*début du parcours*)
    !def , morphisme ;;      

val existe_morphismes :
  int * (int * int) array * bool array ->
  'a * (int * int) array * bool array -> bool * int array = <fun>


In [76]:
let produit aut1 aut2 = 
    let n1, delta1, f1 = aut1 in 
    let n2, delta2, f2 = aut2 in 
    let n = n1 * n2 in 
    let f = Array.make n false in
    let delta = Array.make n (0, 0) in
    let prod_etats n x1 x2 = match x1, x2 with
|(a, b), (c, d) -> n*a + c, n*b + d in
    for i = 0 to n1 - 1 do
        for j = 0 to n2 - 1 do
            begin
            if f1.(i) && f2.(j) then
            f.(n2 * i + j) <- true ;
            delta.(n2 * i + j) <- prod_etats n2 delta1.(i) delta2.(j) ;
            end
        done;
    done;
    partie_accessible (n, delta, f) ;

val produit :
  int * (int * int) array * bool array ->
  int * (int * int) array * bool array ->
  int * (int * int) array * bool array = <fun>
